# XG Boost

this time let's test with XG Boost model.

In [ ]:
# import required libraries
import sys, os
import numpy as np
import pandas as pd
import optuna
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import xgboost as xgb

local_module_path = os.path.abspath(os.path.join(os.path.dirname(os.getcwd()),'preprocess'))
sys.path.append(local_module_path)
from pscript import preprocess

In [ ]:
# load the dataset
train = pd.read_csv("../../../dataset/train.csv")
test = pd.read_csv("../../../dataset/test.csv")

In [ ]:
# preprocess the datasets
ptrain = preprocess(train)
ptest = preprocess(test)

In [ ]:
# ready dataset for training and testing
rtrain = ptrain.drop(['id','accident_risk'], axis=1)
target = ptrain['accident_risk']

rtest = ptest.drop('id', axis=1)

In [ ]:
# split the dataset into training and validation sets
X_train, X_valid, y_train, y_valid = train_test_split(rtrain, target, train_size=0.8, test_size=0.2, random_state=42)

# convert data to dmatrix format
dtrain = xgb.DMatrix(X_train, label=y_train)
dvalid = xgb.DMatrix(X_valid, label=y_valid)

In [ ]:
def objective(trial):
    # Hyperparameter
    params = {
        'objective': 'reg:squarederror',
        'eval_metric': 'rmse',
        'verbosity': 0, # To keep the output clean
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        'max_depth': trial.suggest_int('max_depth', 3, 9),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'lambda': trial.suggest_float('lambda', 1e-8, 10.0, log=True), # L2 regularization
        'alpha': trial.suggest_float('alpha', 1e-8, 10.0, log=True),   # L1 regularization
    }
    
    # training setup
    bst = xgb.train(
        params=params,
        dtrain=dtrain,
        num_boost_round=1000,
        evals=[(dvalid, 'validation')],
        early_stopping_rounds=50,
        verbose_eval=False
    )
    
    # Prediction uses the returned booster (bst)
    preds = bst.predict(dvalid)
    rmse = np.sqrt(mean_squared_error(y_valid, preds))
    
    return rmse

In [ ]:
# Study and optimization
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)

print("--- Optimization Finished ---")
print(f"Best RMSE: {study.best_value}")
print(f"Best params: {study.best_params}")

print("Number of finished trials: ", len(study.trials))
print("Best trial:")
trial = study.best_trial

print(f"  Value (RMSE): {trial.value:.4f}")
print("  Best Hyperparameters: ")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

In [ ]:
# Train the final model with the best hyperparameters
best_params = trial.params
final_model = xgb.XGBRegressor(**best_params)
final_model.fit(rtrain, target)

In [ ]:
fig = optuna.visualization.plot_optimization_history(study)
fig.show()

In [ ]:
# predict the values
preds = final_model.predict(rtest)

In [ ]:
prediction = []
for x in preds:
    prediction.append(round(x,3))

In [ ]:
result = pd.DataFrame({
    'id': ptest['id'],
    'accident_risk': prediction
})

In [ ]:
result.to_csv("../submissions/submission06.csv", index=False)